<a href="https://colab.research.google.com/github/michalcohen/DataAnalysisABC/blob/main/find_invariant.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import os
import pandas as pd
import tqdm
from pandas import DataFrame as df
from os.path import join
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from typing import Dict, List, Tuple
import json
import pandas as pd

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
! pip install --user graphistry[bolt]

In [12]:
import graphistry
graphistry.register(api=3, username='michal', password='dbTrw7BDuQGZ@CF')

In [17]:
f = pd.read_csv("https://raw.githubusercontent.com/michalcohen/DataAnalysisABC/main/pass1.csv")

In [18]:
# Configure graph to use the source/target columns as source/destination of edges
g = graphistry.bind(source='source', destination='target').edges(f)

# Upload & plot
g.plot()